In [1]:
versioninfo()

Julia Version 0.6.4
Commit 9d11f62bcb (2018-07-09 19:09 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-4500U CPU @ 1.80GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell MAX_THREADS=16)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, haswell)


In [2]:
using StatsBase
using IterTools

In [56]:
Pkg.add("LightGraphs")

INFO: Cloning cache of LightGraphs from https://github.com/JuliaGraphs/LightGraphs.jl.git
INFO: Cloning cache of SimpleTraits from https://github.com/mauro3/SimpleTraits.jl.git
INFO: Installing LightGraphs v0.12.0
INFO: Installing SimpleTraits v0.6.0
INFO: Building CodecZlib
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of LightGraphs
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [3]:
function evaluate(solution)
    objective_function = 0
    
    for i in 1:(size(solution)[1]-1)
        objective_function += distances[solution[i],solution[i+1]]
    end

    objective_function += distances[solution[end],solution[1]]
    
    return objective_function
end

function first_sequential_k_opt2(solution)
    #solution_ref = copy(solution)
    #push!(solution_ref, solution[end])
    
    #search
    for i in 1:length(solution)-1
        used_cities = []
        old_weights = distances[solution[i],solution[i+1]]
        new_gain = 0
        
        new_solution = vcat(solution[i:-1:1], solution[end:-1:i+1])
        
        push!(used_cities, solution[i])
        push!(used_cities, solution[i+1])
        for k = 2:5
            found = false
            for city in near_cities[new_solution[end]]
                candidate = findin(new_solution, city)
                if length(candidate) == 1
                    j = candidate[1]
                    if ~(city in used_cities) & ~(new_solution[j+1] in used_cities)
                        if ((old_weights + distances[new_solution[j],new_solution[j+1]]) >
                            (new_gain + distances[new_solution[end], new_solution[j]]
                            + distances[new_solution[1], new_solution[j+1]]))
                            
                            found = true
                            
                            old_weights += distances[new_solution[j],new_solution[j+1]]
                            new_gain += distances[new_solution[end],new_solution[j]]
                            
                            push!(used_cities, new_solution[j])
                            push!(used_cities, new_solution[j+1])
                            
                            new_solution = vcat(new_solution[1:j], new_solution[end:-1:j+1])
                            
                            break
                        end

                    end
                end
            end
            if found == false
                break
            end
        end
        if length(used_cities) > 2
            return new_solution, true
        end 
    end
    #@show solution
    return solution, false 
end

first_sequential_k_opt2 (generic function with 1 method)

In [5]:
#time_limit = Dates.now() + Dates.Second(300)#+ Dates.Second(0.1) #+ Dates.Minute(5)

m = readdlm("TSP_instances/problem_500_100_10000_10000.pctsp");

In [51]:
@time begin
    
n_cities = 100
n_cities_half_up = Int(ceil((n_cities + 1)/2))
n_cities_half_down = Int(floor((n_cities + 1)/2))

init = rand(1:80) # 1 ~ 90

distances = m[(2+init):(init+n_cities+1), init:(init+n_cities-1)]

end

  0.000093 seconds (13 allocations: 78.516 KiB)


100×100 Array{Float64,2}:
    0.0  9684.0  3785.0  6596.0  1528.0  …  3633.0  1836.0  2166.0  8347.0
 9684.0     0.0  3637.0    13.0  1799.0     4785.0  5907.0  1165.0  3386.0
 3785.0  3637.0     0.0  4932.0  5590.0     6360.0  2971.0   513.0  1174.0
 6596.0    13.0  4932.0     0.0  1027.0     5414.0  5175.0   757.0  4475.0
 1528.0  1799.0  5590.0  1027.0     0.0      655.0  2108.0  6467.0  7233.0
 4969.0   306.0  7882.0  3366.0  2110.0  …  9430.0  2511.0  9527.0  2676.0
 2730.0  7705.0  7901.0  9676.0  1777.0     4421.0  9489.0  4950.0  5461.0
 1518.0  6331.0  5027.0  9842.0   998.0     2435.0  1976.0  5163.0  7559.0
 7115.0  9146.0  8968.0  8867.0  1799.0      833.0  5605.0  6931.0   764.0
 8417.0  1545.0  6650.0    45.0   636.0     8428.0  7287.0   493.0  9951.0
 5881.0  4052.0  8524.0  2420.0  4853.0  …  1051.0  9631.0  5149.0   386.0
 4357.0  6221.0  8458.0   403.0  4023.0     3656.0  2393.0  2070.0  7320.0
 5495.0  2013.0  9434.0  8214.0  3224.0      537.0  1038.0  7997.0  5095.0

In [54]:
@time begin

#m=0

max_near_cities = 5

near_cities = Dict()
    
end

# STEP 1 - preprocessing nearest cities for heuristic

@time for i in 1:n_cities
    near_cities[i] = selectperm(distances[i, :], 1:max_near_cities+1)
    shift!(near_cities[i])
end

# STEP 2 - UPPERBOUND (quasi) Lin Kernighan - First improvement sequential 5-opt
@time begin
    upper_bound = 1e10
    a = StatsBase.sample(1:n_cities, n_cities, replace=false, ordered=false)
    best_heuristic_solution = []
    
    for i in 1:100*(n_cities)
        a, better  = first_sequential_k_opt2(a)
        if better == false
            value = evaluate(a)
            if value < upper_bound
                upper_bound = value
                best_heuristic_solution = copy(a)
            end
            a = StatsBase.sample(1:n_cities, n_cities, replace=false, ordered=false)
        end
    end

    value = evaluate(a)
    if value < upper_bound
        upper_bound = value
    end
end

heuristic_upper_bound = upper_bound

  0.000020 seconds (8 allocations: 768 bytes)
  0.000931 seconds (1.42 k allocations: 234.297 KiB)
 10.524150 seconds (60.34 M allocations: 4.829 GiB, 5.90% gc time)


22557.0